Retrieve Data

In [ ]:
#Current weights. Retrieve from csv file
current_weights = {
    "XLK": 0.2934,
    "XLF": 0.1463,
    "XLV": 0.1103,
    "XLY": 0.1017,
    "XLC": 0.0923,
    "XLI": 0.0861,
    "XLP": 0.0657,
    "XLE": 0.0339,
    "XLU": 0.0266,
    "XLRE": 0.0233,
    "XLB": 0.0204,
}


In [ ]:
#ETF quarterly returns
import yfinance as yf
import pandas as pd

tickers = list(current_weights.keys())
start_date = "2018-01-01"

# Download adjusted close prices
data = yf.download(tickers, start=start_date)["Close"]

# Resample to quarterly frequency and compute % returns
quarterly_returns = data.resample("QE").ffill().pct_change().dropna()


[*********************100%***********************]  11 of 11 completed


In [34]:
#ETF Betas
import numpy as np
import statsmodels.api as sm

def calculate_beta(etf_returns, market_returns):
    """Linear regression to estimate beta of each ETF"""
    betas = {}
    for col in etf_returns.columns:
        X = sm.add_constant(market_returns)
        y = etf_returns[col]
        model = sm.OLS(y, X).fit()
        betas[col] = betas[col] = model.params.iloc[1]
    return betas

# Fetch SPY as market proxy
spy = yf.download("SPY", start=start_date)["Close"].resample("QE").ffill().pct_change().dropna()

aligned_returns = quarterly_returns.join(spy["SPY"], how="inner").dropna()
market_returns = aligned_returns["SPY"]
etf_only_returns = aligned_returns.drop(columns="SPY")
# Estimate betas
estimated_betas = calculate_beta(etf_only_returns, market_returns)


[*********************100%***********************]  1 of 1 completed


In [37]:
beta_df = pd.DataFrame.from_dict(estimated_betas, orient='index', columns=['Beta'])
beta_df.index.name = 'ETF'
beta_df = beta_df.sort_values(by='Beta', ascending=False)
display(beta_df)

,Beta
ETF,
XLK,1.218562
XLY,1.183153
XLI,1.101431
XLB,1.073582
XLE,1.055303
XLC,1.052433
XLF,1.011816
XLRE,0.820583
XLV,0.639553


In [ ]:
# -------------------------------------
# 1. Imports
# -------------------------------------
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns

# -------------------------------------
# 2. Load ETF returns
# -------------------------------------
etf_returns = quarterly_returns

# -------------------------------------
# 3. Define current weights (only hardcoded part)
# -------------------------------------

tickers = list(current_weights.keys())
cov_matrix = etf_returns[tickers].cov().values

# -------------------------------------
# 4. Load betas dynamically from file (CSV with columns: ticker, beta)
# -------------------------------------

# -------------------------------------
# 5. Risk-free & ERP input (or pull from config or API)
# -------------------------------------
rf_annual = float(input("Enter annual risk-free rate (e.g., 0.04397): "))
erp_annual = float(input("Enter annual equity risk premium (e.g., 0.0433): "))

# -------------------------------------
# 6. Optimization Function
# -------------------------------------
def optimize_portfolio_weights(tickers, cov_matrix, current_weights, rf_annual, erp_annual, betas, rebalancing_tolerance=0.05):
    w0 = np.array([current_weights[t] for t in tickers])
    rf_q = rf_annual / 4
    erp_q = erp_annual / 4
    mu = np.array([rf_q + betas[t] * erp_q for t in tickers])
    bounds = [(max(0, w - rebalancing_tolerance), min(1, w + rebalancing_tolerance)) for w in w0]
    def neg_sharpe(w, mu, cov, rf):
        port_return = np.dot(w, mu)
        port_vol = np.sqrt(np.dot(w.T, np.dot(cov, w)))
        return -(port_return - rf) / port_vol
    constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
    result = minimize(neg_sharpe, w0, args=(mu, cov_matrix, rf_q), method='SLSQP', bounds=bounds, constraints=constraints)
    optimized = pd.Series(result.x, index=tickers)
    current = pd.Series(current_weights)
    comparison = pd.DataFrame({"Current Weight": current, "Optimized Weight": optimized, "Difference": optimized - current})
    return optimized, comparison, mu, cov_matrix

# -------------------------------------
# 7. Run Optimization
# -------------------------------------
opt_weights, comparison_df, expected_return, cov_matrix = optimize_portfolio_weights(
    tickers, cov_matrix, current_weights, rf_annual, erp_annual, betas
)

print("\nWeight Comparison:")
print(comparison_df.map(lambda x: f"{x:.2%}"))

# -------------------------------------
# 8. Plot Covariance Matrix
# -------------------------------------
cov_df = pd.DataFrame(cov_matrix, index=tickers, columns=tickers)
plt.figure(figsize=(10, 8))
sns.heatmap(cov_df, annot=True, fmt=".5f", cmap="coolwarm", square=True, linewidths=0.5)
plt.title("ETF Quarterly Return Covariance Matrix")
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# -------------------------------------
# 9. Plot Weight Comparison
# -------------------------------------
x = np.arange(len(tickers))
width = 0.35
fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(x - width/2, comparison_df['Current Weight'], width, label='Current', color='skyblue')
ax.bar(x + width/2, comparison_df['Optimized Weight'], width, label='Optimized', color='orange')
ax.set_ylabel('Weight')
ax.set_title('Current vs Optimized ETF Weights')
ax.set_xticks(x)
ax.set_xticklabels(comparison_df.index, rotation=45)
ax.legend()
ax.grid(True, axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()
